# Algorytmika i matematyka uczenia maszynowego 
## Laboratorium 9 - macierz odwrotna i wyznacznik macierzy


### Zadanie 1 - szyfr Hilla

Zaimplementuj dwie funkcje do kodowania i dekodowania wiadomości za pomocą szyfru [Hilla](https://pl.wikipedia.org/wiki/Szyfr_Hilla):
* `encrypt(t, K)` - szyfruje tekst $t$ za pomocą klucza (macierzy) $K$.
* `decrypt(s, K)` - deszyfruje kod $s$ za pomocą klucza (macierzy) $K$.

#### Kodowanie

W systemie kodowania ASCII litery A - Z zapisane są jako liczby z zakresu 65 - 90. Aby zaszyfrować tekst za pomocą klucza $K$ (macierz o wymiarach $m$ x $m$), należy zapisać znaki tekstu w postaci macierzy o wymiarach $m$ x $n$, a następnie wykonać następujące operacje:

1. Utworzyć macierz szyfrującą $K$, której wyznacznik wynosi $det(K) = 1$.
> Uwaga: jest to odstępstwo od oryginalnego algorytmu mające na celu uproszczenie przykładu.
> Macierz taką można utworzyć z macierzy jednostkowej (`np.identity`), korzystając z operacji elementarnych np. dodając do jednego wiersza macierzy inny wiersz pomnożony przez skalar.
1. Zamienić litery tekstu $t$ o długości $h$ na wektor liczb.
1. Dopełnić (_padding_) zerami, aby można było wykonać kolejny krok.
1. Przekształcić na macierz $X$ o wymiarach ($m$ x $n$), gdzie $n = \lceil \frac{h}{m} \rceil$ (możesz użyć funkcji `reshape`).
1. Wykonać operację $S = (KX)$.
1. Skonwertować macierz $S$ na wektor (możesz użyć funkcji `flatten`) i zwrócić szyfrogram $s$ (zaszyfrowany tekst).


> Uwaga 1: Przedstawiony algorytm jest uproszczonym algorytmem, posiadającym ograniczenie $det(K)=1$, które można pominąć, ale wtedy należy do macierzy kodującej wyznaczyć macierz odwrotną modulo 26 (liczba znaków A-Z, ale może być dowolna inna). Podobnie, należy macierz $S$ zamienić na modulo 26. **Istotne**: W tym przypadku należy pamiętać, że wyznacznik macierzy szyfrującej $det(K)$ nie może posiadać wspólnego dzielnika z liczbą 26 (czyli obie liczby muszą być względnie pierwsze). Dlaczego? Bo w przeciwnym wypadku nie istnieje liczba odwrotna do $det(K) \textit{ mod } 26$.

> Uwaga 2: ciąg $s$ może zawierać niedrukowalne znaki. Jeżeli chcesz tego uniknąć możesz np. zmapować znaki (65-90) do zakresu 0-25. Następnie przy wyświetlaniu przeprowadzić operację w drugą stronę.


#### Dekodowanie

Aby rozszyfrować zaszyfrowaną wiadomość $s$, należy:

1. Zamienić ciąg $s$ na macierz $S$.
1. Obliczyć macierz odwrotną $K^{-1}$ (funkcja `np.linalg.inv`).
1. Rozszyfrować wiadomość wykonując operację $W = K^{-1} S$.
1. Skonwertować wiadomość $W$ na ciąg tekstowy $w$.


**Uwaga**: Jest to tylko laboratoryjny przykład na zastosowanie operacji macierzowych. Przedstawione rozwiązanie nie jest bezpieczne.

In [8]:
import numpy as np
import random
def ascii_dic():
    ascii = {}
    for letter in range(ord('A'), ord('Z') + 1):
        ascii[letter] = chr(letter)
    return ascii
ascii = ascii_dic()

def text_to_ascii(text):
    vec = [ord(letter) for letter in text]
    return vec
ascii

{65: 'A',
 66: 'B',
 67: 'C',
 68: 'D',
 69: 'E',
 70: 'F',
 71: 'G',
 72: 'H',
 73: 'I',
 74: 'J',
 75: 'K',
 76: 'L',
 77: 'M',
 78: 'N',
 79: 'O',
 80: 'P',
 81: 'Q',
 82: 'R',
 83: 'S',
 84: 'T',
 85: 'U',
 86: 'V',
 87: 'W',
 88: 'X',
 89: 'Y',
 90: 'Z'}

In [9]:
m = 3
K = np.identity(m)
for i in range(m-1):
    K[0] += random.randint(1, 3) * K[(i+1)]
for j in range(m-1):
    K[j+1] += random.randint(1, 3) * K[0]
print(K)
print(round(np.linalg.det(K), 6))

[[1. 3. 1.]
 [1. 4. 1.]
 [1. 3. 2.]]
1.0


In [10]:
t = 'TOSTERY'

t_list = text_to_ascii(t)
dop = m*m - len(t)
t_list = np.array(t_list + [0]*dop)
X = t_list.reshape(m, int(len(t_list)/m))
print(X)

[[84 79 83]
 [84 69 82]
 [89  0  0]]


In [11]:
S = K @ X
s = S.flatten()
print(s)

[425. 286. 329. 509. 355. 411. 514. 286. 329.]


In [12]:
s = s.reshape(m, int(len(s)/m))
K_inv = np.linalg.inv(K)
W = K_inv @ s
print(W)

[[84. 79. 83.]
 [84. 69. 82.]
 [89.  0.  0.]]


In [13]:
def convert_ascii_to_letters(ascii_numbers):
    output_text = ''
    for i in ascii_numbers:
        if i != 0:  
            output_text += ascii[i]
    return output_text

W = list(W.flatten().astype(int))
print(convert_ascii_to_letters(W))

TOSTERY


### Zadanie 2

Zaimplementować funkcję, która przyjmuje macierz kwadratową jako argument i zwraca jej wyznacznik obliczony zgodnie ze wzorem [Leibniza](https://en.wikipedia.org/wiki/Leibniz_formula_for_determinants) (_definicja permutacyjna_) i porównaj z wynikiem gotowej funkcji z biblioteki numpy `np.linalg.det`.

$$
\text{det}(A) = \sum_{\sigma \in S_n}\left(\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}\right)
$$

, gdzie:

* $S_n$ - [grupa permutacji](https://en.wikipedia.org/wiki/Symmetric_group) (dla macierzy 3x3 będą to permutacje ze zbioru {0, 1, 2})
* $\text{sgn}$ - jest to symbol "+", "-" w zależny od [parzystości permutacji](https://en.wikipedia.org/wiki/Parity_of_a_permutation). Np. dla permutacji `[1, 2, 0]` będzie to "+" (trzeba wykonać dwie operacje - zamienić `0` z `1` a później `1` z `2`, a dla permutacji `[0, 2, 1]` będzie "-" ponieważ wystarczy jedna operacja (zamiana `1` z `2`).
* $\sigma$ - permutacja (element z grupy permutacji $S_n$)

##### Przykład dla macierzy 3x3

| $\sigma$ | $\text{sgn}$ | $\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}$ |
| :---     | :---         | ---: |
| 1, 2, 3  | +            | $+a_{1,1}a_{2,2}a_{3,3}$ |
| 1, 3, 2  | -            | $-a_{1,1}a_{2,3}a_{3,2}$ |
| 3, 1, 2  | +            | $+a_{1,3}a_{2,1}a_{3,2}$ |
| 3, 2, 1  | -            | $-a_{1,3}a_{2,2}a_{3,1}$ |
| 2, 3, 1  | +            | $+a_{1,2}a_{2,3}a_{3,1}$ |
| 2, 1, 3  | -            | $-a_{1,2}a_{2,1}a_{3,3}$ |

$\text{det}(A) = a_{1,1}a_{2,2}a_{3,3} - a_{1,1}a_{2,3}a_{3,2} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} + a_{1,2}a_{2,3}a_{3,1} - a_{1,2}a_{2,1}a_{3,3}$

> Uwaga 1: Aby sprawdzić parzystość permutacji możesz użyć funkcji `parity` z biblioteki [`sympy`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html#sympy.combinatorics.permutations.Permutation.parity). Przykład: 
    ```from sympy.combinatorics import Permutation
Permutation([0, 2, 1]).parity()
    ```
    
> Uwaga 2: W celu wygenerowania permutacji możesz użyć funkcji `permutations` z modułu [`itertools`](https://docs.python.org/3/library/itertools.html#itertools.permutations)

> Uwaga 3: Pamiętaj, że w numpy porównywanie liczb zmiennoprzecinkowych wykonuje się za pomocą funkcji [`allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html)

In [14]:
from itertools import permutations

def sign(perm):
    inversions = 0
    perm = list(perm)
    for i in range(len(perm)):
        for j in range(i + 1, len(perm)):
            if perm[i] > perm[j]:
                inversions += 1
    return -1 if inversions % 2 else 1

def determinant(matrix):
    n = len(matrix)
    perm_list = list(permutations(range(n)))
    det = 0
    for perm in perm_list:
        prod = 1
        for i in range(n):
            prod *= matrix[i][perm[i]]
        det += sign(perm) * prod

    return det

matrix = [
    [1, 2, 3],
    [4, 6, 6],
    [11, 8, 9]
]

print(determinant(matrix))
print(np.linalg.det(matrix))


-36
-36.0
